In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Testing files/TCGAbiolinks/cna_filtered.csv')
train.head()

,bcr_patient_barcode,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,SMIM9,SNORA36A,SNORA56,TMLHE,VBP1,IL9R|ENSG00000124334.12,SPRY3|ENSG00000168939.6,VAMP7|ENSG00000124333.10,WASH6P|ENSG00000182484.10,WASIR1|ENSG00000185203.7
0,TCGA-A8-A075-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,TCGA-A8-A08O-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,TCGA-A8-A0A6-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
3,TCGA-AQ-A04H-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TCGA-BH-A0AZ-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
target=train['bcr_patient_barcode']

In [5]:
train.drop(['bcr_patient_barcode'], axis=1, inplace=True)
train.head()

,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,SMIM9,SNORA36A,SNORA56,TMLHE,VBP1,IL9R|ENSG00000124334.12,SPRY3|ENSG00000168939.6,VAMP7|ENSG00000124333.10,WASH6P|ENSG00000182484.10,WASIR1|ENSG00000185203.7
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_scaled = minmax_scale(train, axis = 0)

In [7]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [8]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

features_path = '/content/drive/MyDrive/AE/deep_autoe_features.pickle'
labels_path = '/content/drive/MyDrive/AE/deep_autoe_labels.pickle'

# this is the size of our encoded representations
encoding_dim = 32

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# save latent space features 32-d vector
# pickle.dump(encoded_imgs, open(features_path, 'wb'))
# pickle.dump(y_test, open(labels_path, 'wb'))

# n = 10  # how many digits we will display
# plt.figure(figsize=(10, 2), dpi=100)
# for i in range(n):
#     # display original
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(x_test[i].reshape(28, 28))
#     plt.gray()
#     ax.set_axis_off()

#     # display reconstruction
#     ax = plt.subplot(2, n, i + n + 1)
#     plt.imshow(decoded_imgs[i].reshape(28, 28))
#     plt.gray()
#     ax.set_axis_off()

# plt.show()

encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(14, 24776)
(2, 24776)
Epoch 1/50
2/2 - 3s - loss: 0.6932 - val_loss: 0.6932 - 3s/epoch - 1s/step
Epoch 2/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 193ms/epoch - 96ms/step
Epoch 3/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 245ms/epoch - 122ms/step
Epoch 4/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 198ms/epoch - 99ms/step
Epoch 5/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 220ms/epoch - 110ms/step
Epoch 6/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 252ms/epoch - 126ms/step
Epoch 7/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 163ms/epoch - 82ms/step
Epoch 8/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 129ms/epoch - 65ms/step
Epoch 9/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 297ms/epoch - 149ms/step
Epoch 10/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 245ms/epoch - 122ms/step
Epoch 11/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 246ms/epoch - 123ms/step
Epoch 12/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 182ms/epoch - 91ms/step
Epoch 13/50
2

In [9]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0.0,1.073349,0.000000,0.373506,0.0,0.057115,0.000000,0.424873,0.000000,0.663284,...,0.0,0.0,0.318650,0.000000,0.000000,0.000000,0.844831,0.000000,0.0,0.0
1,0.0,0.116161,0.000000,0.067141,0.0,0.000000,0.000000,0.212489,0.000000,0.000000,...,0.0,0.0,0.066781,0.000000,0.000000,0.000000,0.217054,0.390451,0.0,0.0
2,0.0,0.000000,0.010603,0.000000,0.0,0.000000,0.191441,0.000000,0.420271,0.000000,...,0.0,0.0,0.000000,0.170986,0.000000,0.000000,0.053322,0.048144,0.0,0.0
3,0.0,0.780580,0.454008,0.133417,0.0,0.000000,0.000000,0.139861,0.998997,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.001445,0.000000,0.318328,0.0,0.0
4,0.0,0.435537,0.000000,0.203765,0.0,0.000000,0.099022,0.000000,0.733509,0.000000,...,0.0,0.0,0.082245,0.000000,0.000000,0.041088,0.458248,0.078468,0.0,0.0
5,0.0,0.311423,0.002612,0.019344,0.0,0.000000,0.186508,0.215286,0.370756,0.386162,...,0.0,0.0,0.000000,0.000000,0.119298,0.000000,0.480098,0.181176,0.0,0.0
6,0.0,0.499375,0.000000,0.233331,0.0,0.000000,0.307455,0.000000,0.570662,0.200954,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.374136,0.343636,0.0,0.0
7,0.0,0.957409,0.145713,0.203245,0.0,0.000000,0.000000,0.592301,0.452638,0.308080,...,0.0,0.0,0.000000,0.212712,0.591518,0.000000,0.746423,0.733237,0.0,0.0
8,0.0,0.334892,0.401568,0.000000,0.0,0.000000,0.114429,0.218189,0.772135,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.083330,0.016500,0.210955,0.0,0.0
9,0.0,0.427334,0.047624,0.293750,0.0,0.000000,0.175444,0.070211,0.627498,0.160465,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.348032,0.791828,0.0,0.0


In [10]:
encoded_train.to_excel('/content/drive/MyDrive/Testing files/TCGAbiolinks/dae_cnv.xlsx', index = False)

In [10]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24776)]           0         
                                                                 
 dense (Dense)               (None, 128)               3171456   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 24776)             319610

In [11]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean()> 0.65])
encoded_train.shape

(16, 21)

In [12]:
#encoded_train.to_excel('/content/drive/MyDrive/Datasets/dae_cnv(proteomics).xlsx', index = False)